This is a Jupyter Notebook that can be used with any book found at Project Gutenberg. However, manual inspection of the book is necessary to check the book display in there. 

The first part of the notebook (A), covers data acquisition. The second part (B), covers cleaning and preprocessing and saving data for later on counting words (so, difficult character removal >> tokenization >> lower casing >> punctuation removal >> stop word removal). The third part (C), covers clearning and preprocessing data for later on applying NER (so, not following the previous pipeline, but just removing noise characters). 

# A. Data Acquisition

### 1. We import the libraries

In [533]:
from urllib import request
from bs4 import BeautifulSoup

import re
import pandas as pd
import string

import nltk 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

### 2. We get the data

In [534]:
url = "https://www.gutenberg.org/cache/epub/76417/pg76417.txt"

In [535]:
response = request.urlopen(url)
raw = response.read().decode("utf-8")

### 3. We prettify the data

In [536]:
soup = BeautifulSoup(raw, "html.parser")

#print(soup.prettify())

In [537]:
data = soup.prettify()
data

'\ufeffThe Project Gutenberg eBook of The children\r\n    \r\nThis ebook is for the use of anyone anywhere in the United States and\r\nmost other parts of the world at no cost and with almost no restrictions\r\nwhatsoever. You may copy it, give it away or re-use it under the terms\r\nof the Project Gutenberg License included with this ebook or online\r\nat www.gutenberg.org. If you are not located in the United States,\r\nyou will have to check the laws of the country where you are located\r\nbefore using this eBook.\r\n\r\nTitle: The children\r\n\r\nAuthor: Edith Wharton\r\n\r\nRelease date: June 29, 2025 [eBook #76417]\r\n\r\nLanguage: English\r\n\r\nOriginal publication: New York: D. Appleton and company, 1928\r\n\r\nCredits: Richard Tonsing and the Online Distributed Proofreading Team at https://www.pgdp.net (This book was produced from images made available by the HathiTrust Digital Library.)\r\n\r\n\r\n*** START OF THE PROJECT GUTENBERG EBOOK THE CHILDREN ***\r\n\r\n\r\n\r\n\r\n\

### 4. We select the data

In [538]:
text = re.search("START OF THE PROJECT GUTENBERG EBOOK THE CHILDREN", data)

In [539]:
text

<re.Match object; span=(874, 923), match='START OF THE PROJECT GUTENBERG EBOOK THE CHILDREN>

In [540]:
text_2 = re.search("END OF THE PROJECT GUTENBERG EBOOK", data)
text_2

<re.Match object; span=(549303, 549337), match='END OF THE PROJECT GUTENBERG EBOOK'>

In [541]:
text_3 = data[923:549303]

In [542]:
text_3

' ***\r\n\r\n\r\n\r\n\r\n\r\n                              THE CHILDREN\r\n\r\n\r\n                             Edith Wharton\r\n\r\n[Illustration: [Logo]]\r\n\r\n                          NEW YORK      LONDON\r\n                        D. Appleton and Company\r\n                               Mcmxxviii\r\n\r\n\r\n\r\n\r\n                         COPYRIGHT — 1928 — BY\r\n                        D. APPLETON AND COMPANY\r\n                PRINTED IN THE UNITED STATES OF AMERICA\r\n\r\n\r\n                   _First Printing, September, 1928_\r\n\r\n\r\n             COPYRIGHT, 1928, BY THE PICTORIAL REVIEW, INC.\r\n\r\n\r\n\r\n\r\n                                   TO\r\n\r\n                          MY PATIENT LISTENERS\r\n\r\n                                   AT\r\n\r\n                             SAINTE-CLAIRE\r\n\r\n\r\n\r\n\r\n                                CONTENTS\r\n\r\n\r\n                                     BOOK I\r\n                                       I\r\n                

In [543]:
text_4 = re.search('As the big liner hung', text_3)
text_4

<re.Match object; span=(2547, 2568), match='As the big liner hung'>

In [544]:
data_1 = text_3[2547:]
data_1

'As the big liner hung over the tugs swarming about her in the bay of\r\nAlgiers, Martin Boyne looked down from the promenade deck on the troop\r\nof first-class passengers struggling up the gangway, their faces all\r\nunconsciously lifted to his inspection.\r\n\r\n“Not a soul I shall want to speak to—as usual!”\r\n\r\nSome men’s luck in travelling was inconceivable. They had only to get\r\ninto a train or on board a boat to run across an old friend; or, what\r\nwas more exciting, make a new one. They were always finding themselves\r\nin the same compartment, or in the same cabin, with some wandering\r\ncelebrity, with the owner of a famous house, of a noted collection, or\r\nof an odd and amusing personality—the latter case being, of course, the\r\nrarest as it was the most rewarding.\r\n\r\nThere was, for instance, Martin Boyne’s own Great-Uncle Edward. Uncle\r\nEdward’s travel-adventures were famed in the family. At home in America,\r\namid the solemn upholstery of his Boston house,

In [545]:
text_5 = re.search("THE END", data_1)
text_5

<re.Match object; span=(544828, 544835), match='THE END'>

In [547]:
data = data_1[:544828]
data

'As the big liner hung over the tugs swarming about her in the bay of\r\nAlgiers, Martin Boyne looked down from the promenade deck on the troop\r\nof first-class passengers struggling up the gangway, their faces all\r\nunconsciously lifted to his inspection.\r\n\r\n“Not a soul I shall want to speak to—as usual!”\r\n\r\nSome men’s luck in travelling was inconceivable. They had only to get\r\ninto a train or on board a boat to run across an old friend; or, what\r\nwas more exciting, make a new one. They were always finding themselves\r\nin the same compartment, or in the same cabin, with some wandering\r\ncelebrity, with the owner of a famous house, of a noted collection, or\r\nof an odd and amusing personality—the latter case being, of course, the\r\nrarest as it was the most rewarding.\r\n\r\nThere was, for instance, Martin Boyne’s own Great-Uncle Edward. Uncle\r\nEdward’s travel-adventures were famed in the family. At home in America,\r\namid the solemn upholstery of his Boston house,

In [548]:
#data = text_3

Let's clean that even more. Let's get rid of all the first part containing the chapter index

# B) Clearning and Pre-Processing. Word counts

This part is very much depending on what we want to do with our data. I have selected to follow the pipeline " difficult character removal >> tokenization >> lower casing >> punctuation removal >> stop word removal". But this is very personal and totally up to us. This cleaning and preprocessing part is taylor-made for counting words (NER will follow a different cleaning and pre-processing process). 

**1. Difficult character removal**

In [549]:
text = re.sub(r'--+', ' ', data)

In [550]:
text_1 = re.sub(r'[‘’“”]', " ", text)

In [551]:
text_1

'As the big liner hung over the tugs swarming about her in the bay of\r\nAlgiers, Martin Boyne looked down from the promenade deck on the troop\r\nof first-class passengers struggling up the gangway, their faces all\r\nunconsciously lifted to his inspection.\r\n\r\n Not a soul I shall want to speak to—as usual! \r\n\r\nSome men s luck in travelling was inconceivable. They had only to get\r\ninto a train or on board a boat to run across an old friend; or, what\r\nwas more exciting, make a new one. They were always finding themselves\r\nin the same compartment, or in the same cabin, with some wandering\r\ncelebrity, with the owner of a famous house, of a noted collection, or\r\nof an odd and amusing personality—the latter case being, of course, the\r\nrarest as it was the most rewarding.\r\n\r\nThere was, for instance, Martin Boyne s own Great-Uncle Edward. Uncle\r\nEdward s travel-adventures were famed in the family. At home in America,\r\namid the solemn upholstery of his Boston house,

**2. Tokenization**

In [552]:
tokens = word_tokenize(text_1)

In [553]:
tokens

['As',
 'the',
 'big',
 'liner',
 'hung',
 'over',
 'the',
 'tugs',
 'swarming',
 'about',
 'her',
 'in',
 'the',
 'bay',
 'of',
 'Algiers',
 ',',
 'Martin',
 'Boyne',
 'looked',
 'down',
 'from',
 'the',
 'promenade',
 'deck',
 'on',
 'the',
 'troop',
 'of',
 'first-class',
 'passengers',
 'struggling',
 'up',
 'the',
 'gangway',
 ',',
 'their',
 'faces',
 'all',
 'unconsciously',
 'lifted',
 'to',
 'his',
 'inspection',
 '.',
 'Not',
 'a',
 'soul',
 'I',
 'shall',
 'want',
 'to',
 'speak',
 'to—as',
 'usual',
 '!',
 'Some',
 'men',
 's',
 'luck',
 'in',
 'travelling',
 'was',
 'inconceivable',
 '.',
 'They',
 'had',
 'only',
 'to',
 'get',
 'into',
 'a',
 'train',
 'or',
 'on',
 'board',
 'a',
 'boat',
 'to',
 'run',
 'across',
 'an',
 'old',
 'friend',
 ';',
 'or',
 ',',
 'what',
 'was',
 'more',
 'exciting',
 ',',
 'make',
 'a',
 'new',
 'one',
 '.',
 'They',
 'were',
 'always',
 'finding',
 'themselves',
 'in',
 'the',
 'same',
 'compartment',
 ',',
 'or',
 'in',
 'the',
 'same',


In [554]:
type(tokens)

list

**3. Lower casing**

In [555]:
lower_tokens = []

for i in tokens:
    lower_tokens.append(i.lower()) 

In [556]:
lower_tokens

['as',
 'the',
 'big',
 'liner',
 'hung',
 'over',
 'the',
 'tugs',
 'swarming',
 'about',
 'her',
 'in',
 'the',
 'bay',
 'of',
 'algiers',
 ',',
 'martin',
 'boyne',
 'looked',
 'down',
 'from',
 'the',
 'promenade',
 'deck',
 'on',
 'the',
 'troop',
 'of',
 'first-class',
 'passengers',
 'struggling',
 'up',
 'the',
 'gangway',
 ',',
 'their',
 'faces',
 'all',
 'unconsciously',
 'lifted',
 'to',
 'his',
 'inspection',
 '.',
 'not',
 'a',
 'soul',
 'i',
 'shall',
 'want',
 'to',
 'speak',
 'to—as',
 'usual',
 '!',
 'some',
 'men',
 's',
 'luck',
 'in',
 'travelling',
 'was',
 'inconceivable',
 '.',
 'they',
 'had',
 'only',
 'to',
 'get',
 'into',
 'a',
 'train',
 'or',
 'on',
 'board',
 'a',
 'boat',
 'to',
 'run',
 'across',
 'an',
 'old',
 'friend',
 ';',
 'or',
 ',',
 'what',
 'was',
 'more',
 'exciting',
 ',',
 'make',
 'a',
 'new',
 'one',
 '.',
 'they',
 'were',
 'always',
 'finding',
 'themselves',
 'in',
 'the',
 'same',
 'compartment',
 ',',
 'or',
 'in',
 'the',
 'same',


**4. Punctuation**

In [557]:
 string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [558]:
punctuation_free = []
punctuation_free = [token for token in lower_tokens if token not in string.punctuation]

In [559]:
punctuation_free

['as',
 'the',
 'big',
 'liner',
 'hung',
 'over',
 'the',
 'tugs',
 'swarming',
 'about',
 'her',
 'in',
 'the',
 'bay',
 'of',
 'algiers',
 'martin',
 'boyne',
 'looked',
 'down',
 'from',
 'the',
 'promenade',
 'deck',
 'on',
 'the',
 'troop',
 'of',
 'first-class',
 'passengers',
 'struggling',
 'up',
 'the',
 'gangway',
 'their',
 'faces',
 'all',
 'unconsciously',
 'lifted',
 'to',
 'his',
 'inspection',
 'not',
 'a',
 'soul',
 'i',
 'shall',
 'want',
 'to',
 'speak',
 'to—as',
 'usual',
 'some',
 'men',
 's',
 'luck',
 'in',
 'travelling',
 'was',
 'inconceivable',
 'they',
 'had',
 'only',
 'to',
 'get',
 'into',
 'a',
 'train',
 'or',
 'on',
 'board',
 'a',
 'boat',
 'to',
 'run',
 'across',
 'an',
 'old',
 'friend',
 'or',
 'what',
 'was',
 'more',
 'exciting',
 'make',
 'a',
 'new',
 'one',
 'they',
 'were',
 'always',
 'finding',
 'themselves',
 'in',
 'the',
 'same',
 'compartment',
 'or',
 'in',
 'the',
 'same',
 'cabin',
 'with',
 'some',
 'wandering',
 'celebrity',
 'wi

**5. Stop words**

In [560]:
stop_words = set(stopwords.words("english"))

clean = [token for token in punctuation_free if token not in stop_words]

super_clean = ' '.join(clean)

In [561]:
super_clean

'big liner hung tugs swarming bay algiers martin boyne looked promenade deck troop first-class passengers struggling gangway faces unconsciously lifted inspection soul shall want speak to—as usual men luck travelling inconceivable get train board boat run across old friend exciting make new one always finding compartment cabin wandering celebrity owner famous house noted collection odd amusing personality—the latter case course rarest rewarding instance martin boyne great-uncle edward uncle edward travel-adventures famed family home america amid solemn upholstery boston house uncle edward model complacent dullness yet whenever got board steamer train _diligence_ distant youth singled fate hero delightful encounter would rachel ill-starred tour states ruskin lake geneva dean canterbury uncle edward appropriate emotions gazing tomb black prince duke devonshire day uncle edward put courteous probably pointless question housekeeper showing chatsworth instantly would receive proscenium box 

**6. Saving things into a text file**

In [562]:
with open("The Children_word_counts.txt", "w", encoding = "utf-8") as f:
    f.write(super_clean)

# C) Cleaning and Pre-Processing. NER

In order to use Spacy pipelines for NER (Name Entity Recognition), we need to keep the text in its oringinal form and only remove noise characters such as r/n/n/r. So lets go back to the data variable and lets do that!

In [563]:
# Remove carriage returns, line feeds, and other escape codes
clean_text = re.sub(r'[\r\n]+', ' ', data)  # replace newlines with a space
clean_text = clean_text.replace('\xa0', ' ')  # remove non-breaking spaces

# Optionally, collapse multiple spaces and strip
clean_text = re.sub(r'\s+', ' ', clean_text).strip()

In [564]:
clean_text

'As the big liner hung over the tugs swarming about her in the bay of Algiers, Martin Boyne looked down from the promenade deck on the troop of first-class passengers struggling up the gangway, their faces all unconsciously lifted to his inspection. “Not a soul I shall want to speak to—as usual!” Some men’s luck in travelling was inconceivable. They had only to get into a train or on board a boat to run across an old friend; or, what was more exciting, make a new one. They were always finding themselves in the same compartment, or in the same cabin, with some wandering celebrity, with the owner of a famous house, of a noted collection, or of an odd and amusing personality—the latter case being, of course, the rarest as it was the most rewarding. There was, for instance, Martin Boyne’s own Great-Uncle Edward. Uncle Edward’s travel-adventures were famed in the family. At home in America, amid the solemn upholstery of his Boston house, Uncle Edward was the model of complacent dullness; ye

In [565]:
with open("The Children_NER.txt", "w", encoding = "utf-8") as f:
    f.write(clean_text)